In [1]:
import numpy as np
import pandas as pd

def load_mnist():

    # 学習データ
    x_train = np.load('../dataset/x_train.npy')
    
    # テストデータ
    x_test = np.load('../dataset/x_test.npy')

    x_train = (x_train.reshape(-1, 784) / 255).astype(np.float32)
    x_test = (x_test.reshape(-1, 784) / 255).astype(np.float32)

    return (x_train, x_test)

x_train, x_test = load_mnist()

import tensorflow as tf
import csv

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

rng = np.random.RandomState(1234)

### define layers ###
tf.reset_default_graph()
z_dim = 10

def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))

def encoder(x):
    with tf.variable_scope('Encoder', reuse=tf.AUTO_REUSE):
        h1 = tf.layers.Dense(units=200, activation=tf.nn.relu)(x)
        h2 = tf.layers.Dense(units=200, activation=tf.nn.relu)(h1)
        mean = tf.layers.Dense(units=z_dim)(h2)
        var = tf.layers.Dense(units=z_dim, activation=tf.nn.softplus)(h2)
    return mean, var

def sampling_z(mean, var):
    epsilon = tf.random_normal(shape=tf.shape(mean))
    z = mean + tf.sqrt(var) * epsilon
    return z

def decoder(z):
    with tf.variable_scope('Decoder', reuse=tf.AUTO_REUSE):
        h3 = tf.layers.Dense(units=200, activation=tf.nn.relu)(z)
        h4 = tf.layers.Dense(units=200, activation=tf.nn.relu)(h3)
        y = tf.layers.Dense(units=784, activation=tf.nn.sigmoid)(h4)
    return y

def lower_bound(x):
    #Encode
    mean, var = encoder(x)
    KL = -0.5 * tf.reduce_mean(tf.reduce_sum(1 + tf_log(var) - mean**2 - var, axis=1))
    
    #Z
    z = sampling_z(mean, var)
    
    #Decode
    y = decoder(z)
    reconstruction = tf.reduce_mean(tf.reduce_sum(x * tf_log(y) + (1 - x) * tf_log(1 - y), axis=1))
    
    lower_bound = [-KL, reconstruction]
    
    return lower_bound


### training ###
#学習データと検証データに分割
x_train, x_valid = train_test_split(x_train, test_size=0.1)

x = tf.placeholder(tf.float32, [None, 784])
lower_bound = lower_bound(x)

cost = -tf.reduce_sum(lower_bound)
optimizer = tf.train.AdamOptimizer()
train = optimizer.minimize(cost)

valid = tf.reduce_sum(lower_bound)

batch_size =100

n_batches = x_train.shape[0] // batch_size
n_epochs = 10

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
#for epoch in range(n_epochs):
for epoch in range(n_epochs):
    rng.shuffle(x_train)
    lower_bound_all = []
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size
        _, lowerbound = sess.run([train, lower_bound], feed_dict={x: x_train[start:end]})
        lower_bound_all.append(lowerbound)
    lower_bound_all = np.mean(lower_bound_all, axis=0)
    lower_bound_valid = sess.run(valid, feed_dict={x: x_valid[0:100]})
    print('EPOCH:%d, Train Lower Bound:%lf, (%lf, %lf), Valid Lower Bound:%lf' %
          (epoch+1, np.sum(lower_bound_all), lower_bound_all[0], lower_bound_all[1], lower_bound_valid))
    
### sampling ###
x = tf.placeholder(tf.float32, [None, 784])
sample_z_func = encoder(x)

z = tf.placeholder(tf.float32, [None, z_dim])
sample_x_func = decoder(z)

# Encode
mean, var = sess.run(sample_z_func, feed_dict={x: x_test})
sample_z = mean

# Decode
sample_x = sess.run(sample_x_func, feed_dict={z: sample_z})


### to_csv ###
with open('../dataset/submission.csv', 'w') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerows(sample_x.reshape(-1, 28*28).tolist())

C:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


EPOCH:1, Train Lower Bound:-283.150330, (-10.693234, -272.457092), Valid Lower Bound:-258.397736
EPOCH:2, Train Lower Bound:-253.788132, (-11.842904, -241.945221), Valid Lower Bound:-250.056213
EPOCH:3, Train Lower Bound:-248.894836, (-12.217196, -236.677643), Valid Lower Bound:-247.416046
EPOCH:4, Train Lower Bound:-246.604431, (-12.332316, -234.272110), Valid Lower Bound:-246.351929
EPOCH:5, Train Lower Bound:-245.134079, (-12.443249, -232.690826), Valid Lower Bound:-244.233551
EPOCH:6, Train Lower Bound:-244.070251, (-12.529029, -231.541229), Valid Lower Bound:-243.031357
EPOCH:7, Train Lower Bound:-243.377258, (-12.643319, -230.733932), Valid Lower Bound:-242.784134
EPOCH:8, Train Lower Bound:-242.741028, (-12.730350, -230.010681), Valid Lower Bound:-242.617722
EPOCH:9, Train Lower Bound:-242.193909, (-12.810193, -229.383713), Valid Lower Bound:-241.881439
EPOCH:10, Train Lower Bound:-241.782104, (-12.884647, -228.897461), Valid Lower Bound:-241.242477
